In [1]:
import os 
import json 
from dotenv import load_dotenv 
from openai import OpenAI  
import gradio as gr 

In [2]:
load_dotenv()

True

In [3]:
MODEL="gpt-4o-mini"
openai=OpenAI()

In [32]:
#MODEL="llama3.2" 
#openai=OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [4]:
system_message = "You are a helpful assistant for an airline called FlightAI" 
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [5]:
def chat(message, history): 
    messages = [{'role':'system', 'content': system_message }] + history + [{'role': "user", 'content': message}] 
    response = openai.chat.completions.create(model=MODEL, messages=messages) 
    return response.choices[0].message.content

Hi There    
Hello! How can I assist you today?     
I want to go to London     
Great choice! Would you like information on flights to London?     
Yes, how much is a ticket     
Ticket prices vary based on dates, class, and availability, please check our website or contact customer service for specific pricing

In [6]:
gr.ChatInterface(fn=chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Tools

In [7]:
ticket_prices = {'london': "$799", 'paris': "$899", 'tokyo': "$1400", 'berlin': "$499"}

In [8]:
def get_ticket_price(destination_city): 
    print(f"Tool get_ticket_price called for {destination_city}")
    return ticket_prices.get(destination_city.lower(), "unknown")

In [9]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [10]:
tools = [{"type": "function", "function": price_function}]

In [11]:
def chatbot(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [12]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

Hi There     
Hello! How can I assist you today?      
I'd like to go to London    
Sure! Would you like to know the ticket price for a return trip to London?     
Yes    
The ticket price for a return to London is $799

In [13]:
gr.ChatInterface(fn=chatbot, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
